# Visualización de Información en Python

> **Autor** \
> Eduardo Graells-Garrido \
> Instituto de Data Science UDD \
> Telefónica I+D Chile \
> egraells@udd.cl \
> @carnby

## Sesión 3: Visualización de Datos Geográficos

> **Caso de Estudio** \
> Data Set: Encuesta de Viajes (Origen-Destino) de Santiago, 2012 \
> Data Set: [Mapas del Censo](http://www.censo2017.cl/servicio-de-mapas/), 2017 \
> Diplomado en Data Science UDD

## Introducción

En este notebook trabajaremos con información geográfica. Seguiremos estudiando el data set de viajes de la encuesta origen-destino, esta vez enfocándonos en distintos patrones geográficos que puedan ayudarnos a responder preguntas específicas.

### Lo que Haremos 

  1. Aprenderemos a cargar y procesar datos geográficos en formato shapefile, con la biblioteca `geopandas`.
  2. Aprenderemos a realizar _dot plots_.
  3. Aprenderemos a realizar _choropleth maps_.

### Preámbulo

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
# incorporamos geopandas! geografía + pandas :)
import geopandas as gpd
from sklearn.preprocessing import normalize

%matplotlib inline
sns.set(context='notebook', font='Fira Sans', style='white', palette='plasma')

In [ ]:
def decode_column(df, fname, col_name, index_col='Id', value_col=None, sep=';', encoding='utf-8', index_dtype=np.float64):
    '''
    param :df: DataFrame del que leeremos una columna.
    param :fname: nombre del archivo que contiene los valores a decodificar.
    param :col_name: nombre de la columna que queremos decodificar.
    param :index_col: nombre de la columna en el archivo @fname que tiene los índices que codifican @col_name
    param :value_col: nombre de la columna en el archivo @fname que tiene los valores decodificados
    param :sep: carácter que separa los valores en @fname. 
    param :encoding: identificación del _character set_ que utiliza el archivo. Usualmente es utf-8, si no funciona,
                     se puede probar con iso-8859-1.
    '''
    if value_col is None:
        value_col = 'value'
        
    values_df = pd.read_csv(fname, sep=sep, index_col=index_col, names=[index_col, value_col], header=0,
                            dtype={index_col: index_dtype}, encoding=encoding)
    
    src_df = df.loc[:,(col_name,)]
    
    return src_df.join(values_df, on=col_name)[value_col]

def normalize_rows(df):
    df = pd.DataFrame(normalize(df, norm='l1'), index=df.index, columns=df.columns)
    return df

def normalize_columns(df):
    df = pd.DataFrame(normalize(df, norm='l1', axis=0), index=df.index, columns=df.columns)
    return df

def z_score_columns(df):
    return df.apply(lambda x: (x - x.mean()) / x.std(), axis=0)

---

## Carga de Datos

Noten que ahora tenemos una variable `path_maps`, que contiene los ficheros con información geográfica en formato `shapefile`.

In [ ]:
path = './EOD_STGO'

In [ ]:
path_maps = './maps'

### Viajes

Primero, carguemos los datos que hemos trabajado en los notebooks anteriores.

In [ ]:
viajes = (pd.read_csv(path + '/viajes.csv', sep=';', decimal=',', index_col='Viaje')
          .join(pd.read_csv(path + '/ViajesDifusion.csv', sep=';', index_col='Viaje'))
          .join(pd.read_csv(path + '/DistanciaViaje.csv', sep=';', index_col='Viaje')))

viajes['ModoDifusion'] = decode_column(viajes, path + '/Tablas_parametros/ModoDifusion.csv', 'ModoDifusion', encoding='latin-1',
                                       index_col='ID')
viajes['Proposito'] = decode_column(viajes, path + '/Tablas_parametros/Proposito.csv', col_name='Proposito')
viajes['SectorOrigen'] = decode_column(viajes, path + '/Tablas_parametros/Sector.csv', 
                                       col_name='SectorOrigen', index_col='Sector', value_col='Nombre', sep=';')
viajes['SectorDestino'] = decode_column(viajes, path + '/Tablas_parametros/Sector.csv', 
                                       col_name='SectorDestino', index_col='Sector', value_col='Nombre', sep=';')
viajes['ComunaOrigen'] = decode_column(viajes, path + '/Tablas_parametros/Comunas.csv', 'ComunaOrigen', 
                                       value_col='Comuna', sep=',')
viajes['ComunaDestino'] = decode_column(viajes, path + '/Tablas_parametros/Comunas.csv', 'ComunaDestino', 
                                       value_col='Comuna', sep=',')
viajes['Periodo'] = decode_column(viajes, path + '/Tablas_parametros/Periodo.csv', 'Periodo', 
                                  sep=';', value_col='Periodos')

# descartamos sectores que no sean relevantes en los orígenes y destinos de los viajes
viajes = viajes[(viajes['SectorOrigen'] != 'Exterior a RM') 
                & (viajes['SectorDestino'] != 'Exterior a RM')
                & (viajes['SectorOrigen'] != 'Extensión Sur-Poniente') 
                & (viajes['SectorDestino'] != 'Extensión Sur-Poniente')
                & pd.notnull(viajes['SectorOrigen'])
                & pd.notnull(viajes['SectorDestino'])
                # también descartamos viajes que hayan sido imputados en la encuesta
                & (viajes['Imputada'] == 0)
                # y finalmente descartamos viajes cuya distancia indique que son viajes cortísimos o bien demasiado largos para el tamaño de la ciudad
                & (viajes['DistManhattan'].between(500, 35000))]

print(len(viajes))

Usaremos la misma codificación personalizada para los propósitos de viaje:

In [ ]:
from collections import defaultdict

propositos_agregados = defaultdict(lambda: 'Otro')

propositos_agregados.update({
    'Al estudio': 'Estudio',
    'Al trabajo': 'Trabajo',
    'Por estudio': 'Estudio',
    'Por trabajo': 'Trabajo',
    'volver a casa': 'Volver a Casa',
    'De salud': 'Necesidades',
    'De compras': 'Necesidades',
    'Trámites': 'Necesidades'
})

viajes['PropositoAgregado'] = viajes['Proposito'].map(lambda x: propositos_agregados[x])

Cargamos la tabla de personas porque utilizaremos su factor de expansión:

In [ ]:
personas = pd.read_csv(path + '/personas.csv', sep=';', decimal=',', encoding='utf-8')

In [ ]:
viajes_persona = viajes.merge(personas)

In [ ]:
viajes_persona['PesoLaboral'] = viajes_persona['FactorLaboralNormal'] * viajes_persona['Factor']

Así, el total de viajes que tenemos para un día laboral es:

In [ ]:
print('{} viajes expandidos a {}'.format(len(viajes_persona), int(viajes_persona['PesoLaboral'].sum())))

### Geografía

Para datos geográficos trabajaremos con el formato `shapefile`. Se define así:

> El formato ESRI Shapefile (SHP) es un formato de archivo informático propietario de datos espaciales desarrollado por la compañía ESRI, quien crea y comercializa software para Sistemas de Información Geográfica como Arc/Info o ArcGIS. Originalmente se creó para la utilización con su producto ArcView GIS, pero actualmente se ha convertido en formato estándar de facto para el intercambio de información geográfica entre Sistemas de Información Geográfica por la importancia que los productos ESRI tienen en el mercado SIG y por estar muy bien documentado.
>
> Un shapefile es un formato vectorial de almacenamiento digital donde se guarda la localización de los elementos geográficos y los atributos asociados a ellos. No obstante carece de capacidad para almacenar información topológica. Es un formato multiarchivo, es decir está generado por varios ficheros informáticos. 
>
> -- [Wikipedia](https://es.wikipedia.org/wiki/Shapefile)

En general, los archivos `.shp` se almacenan en una carpeta donde otros archivos complementarios a la geografía tienen el mismo nombre pero con extensiones diferentes, por ejemplo, para almacenar meta-datos de la geografía.

#### Shapefile de la Encuesta Origen-Destino

La encuesta origen-destino disponibiliza un `shapefile` de las zonas de estudio en la [biblioteca](http://www.sectra.gob.cl/encuestas_movilidad/encuestas_movilidad.htm) de la SECTRA (Secretaría de Transporte). Sin embargo, el link está escondido en el código fuente de la página, ya que presenta problemas de rendering. Afortunadamente ya está en el repositorio del curso :)

Para cargar `shapefiles` podemos usar la función `read_file` de `geopandas`, entregando la carpeta donde esté nuestro archivo `.shp`. Si solamente hay un `.shp`, la biblioteca lo detecta automáticamente:

In [ ]:
zonas_eod = gpd.read_file(path_maps + '/Zonificacion_EOD2012')
zonas_eod.head()

Como ven, su apariencia es la de un `DataFrame`. Sin embargo, la columna `geometry` tiene un significado especial: contiene los elementos geográficos.

Adicionalmente, un `GeoDataFrame` tiene un método `plot` que se encarga de graficar los contenidos geográficos:

In [ ]:
zonas_eod.plot()

El sistema de coordenadas utilizado por este fichero se puede ver con el atributo `crs` del `GeoDataFrame`:

In [ ]:
zonas_eod.crs

Aquí está la especificación del sistema de coordenadas [`EPSG:32719`](https://epsg.io/32719). Más adelante veremos que los archivos pueden tener sistemas de coordenadas distintos, pero que `geopandas` nos ayuda a trabajar con ello.

Un `GeoDataFrame` puede hacer las mismas operaciones que un `DataFrame` tradicional, incluyendo filtrado utilizando el operador `[]`. Aquí graficaremos las zonas para las cuales tenemos viajes en nuestra tabla de viajes. Noten que la columna `ID` del `GeoDataFrame` tiene (por definición) la misma codificación que las columnas `ZonaOrigen` y `ZonaDestino` de la tabla de viajes:

In [ ]:
zonas_con_viajes = zonas_eod[zonas_eod.ID.isin(viajes.ZonaOrigen.unique())
                           & zonas_eod.ID.isin(viajes.ZonaDestino.unique())]
zonas_con_viajes.plot()

Tenemos menos zonas. Sin embargo, queda la sensación de que tenemos muchas zonas grandes correspondientes a zonas rurales que ocupan más espacio en el mapa que las zonas urbanas. Podemos mejorar esto utilizando datos del Censo 2017.

#### Shapefiles del Censo

El Censo 2017 disponibiliza una serie de `shapefiles` para cada región del país. En este caso, cargaremos un mapa para la Región Metropolitana (en la carpeta `R13`) con los límites urbanos definidos por el INE. Noten que en este caso le indicamos el nombre del archivo `.shp` a `geopandas`, debido a que la carpeta tiene otros ficheros `.shp`.

In [ ]:
rm = gpd.read_file(path_maps + '/R13/LIMITE_URBANO_CENSAL_C17.shp')

In [ ]:
rm.head()

In [ ]:
rm.plot()

Notamos que tiene una área geográfica por comuna. Sabemos que en Santiago hay tres provincias que nos interesan, así que nos gustaría tener un área por provincia (que podría tener varios polígonos, no necesariamente uno). Para ello podemos usar el método `dissolve` de `geopandas` de la siguiente manera:

In [ ]:
provincias = rm.dissolve(by='NOM_PROVIN')
provincias.plot()

In [ ]:
provincias.head()

Como ven, desaparecieron los bordes comunales, y la columna `NOM_PROVIN` define un índice en la tabla. Esto se debe a que el resultado es equivalente a hacer un `groupby` en `pandas`. Con el operador `.loc[]` de `pandas` (y por ende, `geopandas`) podemos elegir las filas del índice que nos interesan: 

In [ ]:
provincias = provincias.loc[['SANTIAGO', 'CORDILLERA', 'MAIPO']]
provincias.plot()

¡Mucho mejor! El siguiente paso es ver como podemos intersectar las zonas EOD que tenemos con estas áreas urbanas, de modo que las zonas que estén en los bordes urbanos se recorten y queden solamente con la sección urbana que les corresponda.

Para ello podemos usar la operación `overlay` de `geopandas`. Este método recibe dos `GeoDataFrame` y un nombre de operación. En este caso, diremos que queremos _intersectar_ con `zonas_con_viajes` y `rm`:

In [ ]:
zonas_urbanas = gpd.overlay(zonas_con_viajes, rm, how='intersection')
zonas_urbanas.plot()

¡No pasó nada! 

Omitimos un paso crucial a la hora de trabajar con distintos `GeoDataFrames`: verificar que tengan el mismo sistema de coordenadas.

In [ ]:
rm.crs

No eran el mismo. Podemos usar el método `to_crs` de `geopandas` para cambiar el sistema de coordenadas de un `geodataframe`:

In [ ]:
zonas_urbanas = gpd.overlay(zonas_con_viajes.to_crs(rm.crs), rm, how='intersection')
zonas_urbanas.plot()

¡Ahora sí! Tenemos las mismas zonas EOD de antes, pero en el sistema de coordenadas de `rm`, y considerando solamente las áreas urbanas definidas en el censo. Así quedó el `GeoDataFrame` final:

In [ ]:
zonas_urbanas.head()

El siguiente paso es cambiarle el índice, eligiendo el atributo `ID` para ese fin. Esto nos permitirá cruzarlo con la tabla de viajes si es necesario.

In [ ]:
zonas_urbanas = zonas_urbanas.set_index('ID')

En este punto, ya tenemos dos elementos que podemos utilizar para definir tareas:

- La tabla de viajes, cruzada con la tabla de personas.
- Geografía de Santiago utilizando la misma unidad de análisis espacial: zonas EOD.

Definamos _tareas_ o _preguntas a responder_ con estos datos.

---

## 1. ¿Cuál es la distribución geográfica de los viajes de acuerdo a la cantidad de combinaciones que realizan?

Para responder esta pregunta consideremos lo siguiente:

- Nos interesa particularmente el _origen_ de los viajes. Por ej., esperaríamos que una comuna más periférica tuviese viajes con más combinaciones que una comuna céntrica. La tabla de viajes contiene las columnas `OrigenCoordX` y `OrigenCoordY` que nos permitirán analizar este punto.
- Nos interesa el patrón geográfico de la distribución (no solamente saber que periférica => más combinaciones). Por tanto, necesitamos un mapa.
- La cantidad de combinaciones está implícitamente codificada en la variable `Etapas` de la tabla de viajes.

In [ ]:
viajes_persona.columns

¿Cómo lucen las coordenadas?

In [ ]:
viajes_persona[['OrigenCoordX', 'OrigenCoordY']].head()

Si bien las coordenadas están en formato numérico, no sabemos en qué sistema de coordenadas están, ni tampoco las tenemos estructuradas en un `GeoDataFrame`. Para ello, usaremos la función `points_from_xy` de `geopandas` para darle contexto geográfico a los datos. Supondremos que las coordenadas están en el mismo sistema de referencia que el `shapefile` de zonas EOD.

In [ ]:
origenes_viajes = gpd.GeoDataFrame(viajes_persona,
                                   geometry=gpd.points_from_xy(viajes_persona['OrigenCoordX'], viajes_persona['OrigenCoordY']),
                                   crs=zonas_eod.crs)
print(len(origenes_viajes))
origenes_viajes.head()

Grafiquemos los orígenes de los viajes en conjunto con las zonas:

In [ ]:
ax = zonas_eod.plot(figsize=(12, 12), color='#efefef', edgecolor='#abacab', linewidth=1)
origenes_viajes.plot(ax=ax, markersize=1, marker='.')

In [ ]:
origenes_urbanos = gpd.sjoin(origenes_viajes.to_crs(zonas_urbanas.crs), 
                            zonas_urbanas, 
                            op='within', lsuffix='_l', rsuffix='_r')
print(len(origenes_urbanos))

In [ ]:
origenes_urbanos.geometry

In [ ]:
ax = zonas_urbanas.plot(figsize=(12, 12), color='#efefef', edgecolor='#abacab', linewidth=1)
(origenes_urbanos
 [origenes_urbanos.PropositoAgregado == 'Trabajo']
 .plot(categorical=True, 
       column='Etapas', 
       ax=ax, 
       marker='.', 
       markersize=10,  
       cmap='magma_r', 
       legend=True))

ax.set_axis_off()

In [ ]:
from sklearn.preprocessing import minmax_scale
origenes_urbanos['PesoVisual'] = minmax_scale(origenes_urbanos['PesoLaboral'].values, (0.01, 1.0))

In [ ]:
ax = zonas_urbanas.plot(figsize=(12, 12), color='#efefef', edgecolor='#abacab', linewidth=1)
origenes_a_graficar = origenes_urbanos[origenes_urbanos.PropositoAgregado == 'Trabajo']

(origenes_a_graficar
 .plot(categorical=True, 
       column='Etapas', 
       ax=ax, 
       marker='o', 
       markersize=origenes_a_graficar['PesoVisual'] * 500,  
       alpha=0.75,
       cmap='magma_r', 
       legend=True))

ax.set_axis_off()

In [ ]:
n_columns = 2
n_rows = 2

# Create figure and axes (this time it's 9, arranged 3 by 3)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_columns, figsize=(9 * n_columns, 9 * n_rows))
# Make the axes accessible with single indexing
axes = axes.flatten()

for ax, i in zip(axes, range(1, 5)):
    zonas_urbanas.plot(ax=ax, color='#efefef', edgecolor='#abacab', linewidth=1)
    ax.set_title('{} Etapa(s)'.format(i))
    
    origenes_a_graficar = (origenes_urbanos
                          [(origenes_urbanos.PropositoAgregado == 'Trabajo') &
                           (origenes_urbanos.Etapas == i)])
    
    (origenes_a_graficar
     .plot(categorical=True, 
           column='Etapas', 
           ax=ax, 
           marker='o', 
           markersize=origenes_a_graficar['PesoVisual'] * 500,
           alpha=0.75,
           legend=False))
    ax.set_axis_off()
    
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
colors = sns.color_palette('magma_r', n_colors=4)
sns.palplot(colors)

In [ ]:
n_columns = 2
n_rows = 2

# Create figure and axes (this time it's 9, arranged 3 by 3)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_columns, figsize=(9 * n_columns, 9 * n_rows))
# Make the axes accessible with single indexing
axes = axes.flatten()

for ax, i, color in zip(axes, range(1, 5), colors):
    zonas_urbanas.plot(ax=ax, color='#efefef', edgecolor='#abacab', linewidth=1)
    
    origenes_a_graficar = (origenes_urbanos
                          [(origenes_urbanos.PropositoAgregado == 'Trabajo') &
                           (origenes_urbanos.Etapas == i)])
    
    ax.set_title('{} Etapa(s): {} Viajes'.format(i, int(origenes_a_graficar['PesoLaboral'].sum())))
    
    (origenes_a_graficar
     .plot(categorical=True, 
           column='Etapas', 
           ax=ax, 
           marker='o', 
           markersize=origenes_a_graficar['PesoVisual'] * 500,
           alpha=0.75,
           legend=False,
           edgecolor='black',
           linewidth=0.5,
           color=color))
    ax.set_axis_off()
    
plt.subplots_adjust(wspace=0, hspace=0)

## 2. ¿Cuán lejos queda el trabajo de acuerdo al lugar de residencia?

In [ ]:
viajes_trabajo = viajes_persona[(viajes_persona.PropositoAgregado == 'Trabajo') &
                                (pd.notnull(viajes_persona.PesoLaboral))]
print(len(viajes_trabajo), viajes_trabajo.PesoLaboral.sum())

In [ ]:
viajes_trabajo.columns

In [ ]:
viajes_trabajo['DistEuclidiana'].mean()

In [ ]:
def weighted_mean(df, value='DistEuclidiana', weight='PesoLaboral'):
    weighted_sum = (df[value] * df[weight]).sum()
    return weighted_sum / df[weight].sum()

weighted_mean(viajes_trabajo)

In [ ]:
distancia_zonas = viajes_trabajo.groupby(['ZonaOrigen']).apply(weighted_mean)

In [ ]:
distancia_zonas

In [ ]:
distancia_zonas.name = 'distancia_al_trabajo'

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 1, figsize=(12, 12))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

zonas_urbanas.join(distancia_zonas).plot(ax=ax, cax=cax,
                                         column='distancia_al_trabajo', 
                                         cmap='YlGnBu',
                                         edgecolor='#abacab',
                                         linewidth=0.5,
                                         legend=True)

ax.set_axis_off()
cax.set_ylabel('Distancia')
ax.set_title('Promedio de Distancia al Trabajo en Santiago')

## 3. ¿Dónde se utiliza cada modo de transporte para viajes al trabajo/estudio?

In [ ]:
viajes_por_modo = (viajes_persona
                   [viajes_persona.PropositoAgregado.isin(['Trabajo'])]
                   .groupby(['ZonaOrigen', 'ModoDifusion'])
                   .agg(n_viajes=('PesoLaboral', 'sum'))
                   ['n_viajes'].unstack(fill_value=0)
                   .pipe(normalize_columns))
viajes_por_modo.head()

In [ ]:
zonas_urbanas.join(viajes_por_modo).plot(column='Auto', cmap='magma_r', legend=True)

In [ ]:
zonas_con_modo = zonas_urbanas.join(viajes_por_modo)

In [ ]:
n_columns = 4
n_rows = int(np.ceil(len(viajes_por_modo.columns) / n_columns))

# Create figure and axes (this time it's 9, arranged 3 by 3)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_columns, figsize=(4 * n_columns, 6 * n_rows))
# Make the axes accessible with single indexing
axes = axes.flatten()

for ax, column in zip(axes, viajes_por_modo.columns):
    zonas_con_modo.plot(column=column, cmap='magma_r', legend=False, ax=ax, edgecolor='#abacab', linewidth=0.5, k=10)
    ax.set_title(column)
    ax.set_axis_off()
    
for ax in axes[len(viajes_por_modo.columns):]:
    ax.set_visible(False)
    
plt.subplots_adjust(wspace=0, hspace=0)

## 4. ¿Cuál es la distribución geográfica de las actividades en la ciudad?

In [ ]:
viajes_por_proposito = (viajes_persona
                   .groupby(['ZonaDestino', 'PropositoAgregado'])
                   .agg(n_viajes=('PesoLaboral', 'sum'))
                   ['n_viajes'].unstack(fill_value=0)
                   #.pipe(normalize_columns)
                  )
viajes_por_proposito.head()

In [ ]:
zonas_con_proposito = zonas_urbanas.join(viajes_por_proposito)

In [ ]:
ax = zonas_con_proposito.plot(column='Trabajo', cmap='inferno_r', legend=True, scheme='FisherJenks', k=10)
ax.get_legend().set_bbox_to_anchor((1., 0., 0.75, 0.9))
plt.axis('off')
plt.xlabel('Fracción de los Viajes')

In [ ]:
n_columns = 3
n_rows = int(np.ceil(len(viajes_por_proposito.columns) / n_columns))

# Create figure and axes (this time it's 9, arranged 3 by 3)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_columns, figsize=(4 * n_columns, 6 * n_rows))
# Make the axes accessible with single indexing
axes = axes.flatten()

for ax, column in zip(axes, viajes_por_proposito.columns):
    zonas_con_proposito.plot(column=column, cmap='magma_r', legend=False, ax=ax, edgecolor='#abacab', linewidth=0.5, k=10,
                            scheme='FisherJenks')
    ax.set_title(column)
    ax.set_axis_off()
    
for ax in axes[len(viajes_por_proposito.columns):]:
    ax.set_visible(False)
    
plt.subplots_adjust(wspace=0, hspace=0)